# Intro

This notebook is to demonstrate how to convert a [libpgm](https://github.com/CyberPoint/libpgm) discrete Bayesian Belief Network (BBN) into a py-bbn BBN one. The JSON data specified [here](https://pythonhosted.org/libpgm/unittestdict.html) is the example taken.

In [1]:
json_data = {
    "V": ["Letter", "Grade", "Intelligence", "SAT", "Difficulty"],
    "E": [["Intelligence", "Grade"],
        ["Difficulty", "Grade"],
        ["Intelligence", "SAT"],
        ["Grade", "Letter"]],
    "Vdata": {
        "Letter": {
            "ord": 4,
            "numoutcomes": 2,
            "vals": ["weak", "strong"],
            "parents": ["Grade"],
            "children": None,
            "cprob": {
                "['A']": [.1, .9],
                "['B']": [.4, .6],
                "['C']": [.99, .01]
            }
        },

        "SAT": {
            "ord": 3,
            "numoutcomes": 2,
            "vals": ["lowscore", "highscore"],
            "parents": ["Intelligence"],
            "children": None,
            "cprob": {
                "['low']": [.95, .05],
                "['high']": [.2, .8]
            }
        },

        "Grade": {
            "ord": 2,
            "numoutcomes": 3,
            "vals": ["A", "B", "C"],
            "parents": ["Difficulty", "Intelligence"],
            "children": ["Letter"],
            "cprob": {
                "['easy', 'low']": [.3, .4, .3],
                "['easy', 'high']": [.9, .08, .02],
                "['hard', 'low']": [.05, .25, .7],
                "['hard', 'high']": [.5, .3, .2]
            }
        },

        "Intelligence": {
            "ord": 1,
            "numoutcomes": 2,
            "vals": ["low", "high"],
            "parents": None,
            "children": ["SAT", "Grade"],
            "cprob": [.7, .3]
        },

        "Difficulty": {
            "ord": 0,
            "numoutcomes": 2,
            "vals": ["easy", "hard"],
            "parents": None,
            "children": ["Grade"],
            "cprob":  [.6, .4]
        }
    }
}

Here we kludge how to build a libpgm discrete network. Note that libpgm requires node data and a skeleton. Since I could not find a factory method to read in from a dictionary or JSON string (they only show how to read the JSON data from a file), I took a look at the code and manually constructed a discrete BBN in libpgm. If the libpgm API changes, then this all might break too. 

In [2]:
from libpgm.nodedata import NodeData
from libpgm.graphskeleton import GraphSkeleton
from libpgm.discretebayesiannetwork import DiscreteBayesianNetwork

nd = NodeData()
nd.Vdata = json_data["Vdata"]

skel = GraphSkeleton()
skel.V = json_data["V"]
skel.E = json_data["E"]
skel.toporder()

bn = DiscreteBayesianNetwork(skel, nd)

After you have the libpgm BBN, you can use the `Factory` to create a py-bbn BBN and perform exact inference as follows.

In [3]:
from pybbn.graph.dag import Bbn
from pybbn.graph.jointree import EvidenceBuilder
from pybbn.pptc.inferencecontroller import InferenceController
from pybbn.graph.factory import Factory

bbn = Factory.from_libpgm_discrete_object(bn)
join_tree = InferenceController.apply(bbn)

These are the marginal probabilities of each node.

In [4]:
# print the marginal probabilities
for node in join_tree.get_bbn_nodes():
    potential = join_tree.get_bbn_potential(node)
    print(node)
    print(potential)
    print('>')

0|Difficulty|easy,hard
0=easy|0.6
0=hard|0.4
>
1|Intelligence|low,high
1=low|0.7
1=high|0.3
>
2|Grade|A,B,C
2=A|0.362
2=B|0.2884
2=C|0.3496
>
3|Letter|weak,strong
3=weak|0.497664
3=strong|0.502336
>
4|SAT|lowscore,highscore
4=lowscore|0.725
4=highscore|0.275
>


These are the marginal probabilities given that we observe `SAT=highscore`.

In [5]:
# insert an observation evidence
ev = EvidenceBuilder() \
    .with_node(join_tree.get_bbn_node_by_name('SAT')) \
    .with_evidence('highscore', 1.0) \
    .build()
join_tree.set_observation(ev)

# print the marginal probabilities
for node in join_tree.get_bbn_nodes():
    potential = join_tree.get_bbn_potential(node)
    print(node)
    print(potential)
    print('>')

0|Difficulty|easy,hard
0=easy|0.6
0=hard|0.4
>
1|Intelligence|low,high
1=low|0.127272727273
1=high|0.872727272727
>
2|Grade|A,B,C
2=A|0.671272727273
2=B|0.189890909091
2=C|0.138836363636
>
3|Letter|weak,strong
3=weak|0.280531636364
3=strong|0.719468363636
>
4|SAT|lowscore,highscore
4=lowscore|0.0
4=highscore|1.0
>


These are the marginal probabilities given that we observe `SAT=lowscore`.

In [6]:
# insert an observation evidence
ev = EvidenceBuilder() \
    .with_node(join_tree.get_bbn_node_by_name('SAT')) \
    .with_evidence('lowscore', 1.0) \
    .build()
join_tree.set_observation(ev)

# print the marginal probabilities
for node in join_tree.get_bbn_nodes():
    potential = join_tree.get_bbn_potential(node)
    print(node)
    print(potential)
    print('>')

0|Difficulty|easy,hard
0=easy|0.6
0=hard|0.4
>
1|Intelligence|low,high
1=low|0.91724137931
1=high|0.0827586206897
>
2|Grade|A,B,C
2=A|0.244689655172
2=B|0.325765517241
2=C|0.429544827586
>
3|Letter|weak,strong
3=weak|0.580024551724
3=strong|0.419975448276
>
4|SAT|lowscore,highscore
4=lowscore|1.0
4=highscore|0.0
>
